## Guribo功能和操作进阶

## 重要参数和属性

1. 参数主要是控制优化器的行为，需要在优化启动前设置。例如：控制求解时间TimeLimit、控制控制台输出log记录LogToConsole。

2. 属性：控制模型(模型、变量、约束、目标等对象)的特征。例如：模型ModelSense来调整模型的优化方向，比如说最大化、最小化。变量LB/UB、约束RHS。

### 参数

Guribo的参数主要分为三大类，差不多130多个参数。

1. Termination: 停止参数，控制求解器停止条件。例如TimeLimit设定求解时间，SolutionLimit设定MIP可行解数量。
2. Tolerances: 容差参数，控制求解器的最优或可行的偏差。例如MIPGap设定MIP的gap值，一旦Guribo达到设定的Gap值的时候，就会终止。FeasibilityTol设定模型的精度。
3. Simplex: 单纯形参数，控制单纯形法。例如InfUbbdInfo控制是否获取不可行或无界模型的额外信息。
4. Barrier: 障碍法参数，控制障碍法，例如QCPDual控制是否获取二次模型的对偶值。
5. MIP混合整数规划参数，控制混合整数规划算法。例如BranchDir设定有限分支方向：Heuristic设定启发式算法求解时间所占的比重。
6. Mip Cuts: 割平面参数，控制割平面，例如：Cuts设定割平面法的强度。
7. Tuning: 调参参数，控制调参工具。例如：TuneCriterion设定参数的准则：TuneTimeLimit设定调参的时间。
8. Multiple Solution: 多解参数，尝试寻找多个解，例如PoolSolutions决定存储可行解的数量。

9. Distributed algorithms: 分布式计算参数。
10. Compute Server: 计算服务器参数。
11. Cloud: 云参数。
12. Token server: 令牌服务器参数。
13. Other: 其他一些参数。

### 参数设置方法

主要是用过setParam(paramname, newvalue)来设置，paramname参数名称，newvalue参数取值，可以设定为“default”。

对于python而言，可以简写为model.Params.xxx。例如，设定求解时间：

```python
model.setParam("TimeLimit", 600)
model.setParam(GRB.Param.TimeLimit, 600)
model.Params.TimeLimit = 600
```

上述3种方法是等价的。

这里着重介绍几个参数：

|参数名称|作用|取值|
|:-|:-|:-|
|TimeLimit|时间设定|单位秒|
|MIPFocus|设定MIP的求解侧重点|0:默认, 均衡搜寻可行解和证明最优; <br> 1:侧重快速找到可行解; <br>2:侧重证明最优; <br>3:侧重界提升|
|LogToConsole|log记录是否在控制台显示|0:关闭控制台显示; <br>1:默认,打开控制台显示|
|LogFile|设定log文件名称|默认空字符串|
|MIPGap|设gap值|默认0.0001, 小于给定值终止计算|
|ObjNumber|目标函数索引(多目标)|索引从0开始编号|


- `MIPFocus`: 设定`MIP`的求解侧重点。参数的含义如下: `0`默认，均衡搜寻可行解和证明最优; `1`侧重证明最优; `2`侧重界的提升(如果发现界提升缓慢的话)。

- `Method`: 设定线性模型或MIP根节点的求解方法。`-1`默认，(3, LP, 2 QP/QCP, 1 MIP); `0`初始单纯形; `1`对偶单纯形; `2`内点法; `3` (0,1, 2)并发(线程数，重复运行最优基可能不同); `4` 确定性(0, 1, 2)并发，重复运行结果相同。`5` 确定性(0,1)并发，重复运行结果相同。

- `LogToConsole`: log记录是否在控制台显示。0关闭控制台显示，1打开控制台显示(默认)。

- `LogFile`: 设定log文件名称，默认空字符串。

- `Presolve`: 控制预处理的程度。-1默认，0关闭预处理，1保守倾向。

- `MIPGap`: 设定Gap，0，0001默认，小于给定的值终止计算。

- `ImproveStartGap`: 提升策略开始gap值。0.0默认，例如取0.1，当gap<0.1,开始提升策略。

- `ImproveStartNodes`: 提升策略开始节点值，Infinity默认，例如取5，当节点数>5，开始提升策略。

- `ImproveStartTime`: 提升策略开始的时间，Infinity默认，例如取100，当运行时间到100s时，开始提升策略。

- `ObjNumber`: 目标函数索引(多目标)，索引从0开始编号。

In [1]:
# Example
# Import Envs
import sys
import gurobipy as gp
from gurobipy import GRB

# Read model
m = gp.read('net12.lp')

# Set a 2 second time limit
# Below are equivalent
# model.setParam("TimeLimit", 600)
# model.setParam(GRB.Param.TimeLimit, 600)
# model.Params.TimeLimit = 600
m.setParam(GRB.Param.TimeLimit, 2)  # 最大时间限制为2s。

# solve the model with diff values of MIPFocus
bestModel = m.copy()
bestModel.optimize()


# 循环挑选MIPFocus的值，取值范围为1，2，3。
for i in range(1, 4):
    # .reset() discard any solution information
    m.reset()
    m.Params.MIPFocus = i
    m.optimize()
    if bestModel.MIPGap > m.MIPGap:  # 发现了更好的MIPGap的值的时候，更新模型。
        bestModel, m = m, bestModel
        
# Set timeLimit to inf and print the bestModel resuslt
del m
bestModel.Params.timeLimit = 'default'
bestModel.optimize()
print('Solved with MIPFocus: %d' % bestModel.Params.MIPFocus)

Restricted license - for non-production use only - expires 2022-01-13
Read LP format model from file net12.lp
Reading time = 0.05 seconds
R14022: 14021 rows, 14115 columns, 80384 nonzeros
Changed value of parameter TimeLimit to 2.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

### 属性

&emsp;&emsp;属性类别, 共10大类: 

- `Model Attributes`; 模型属性
    - ModelSense; 模型优化方向(最大化/最小化), gurobi默认为最小化。
    - ObjVal; 当前目标值
- `Variable Attributes`; 变量属性，这个属性主要用于求解完成后，查询变量的取值。
    - Start; 初始解(给一个质量比较好的初始解), 这个出世解可以给所有的解，也可以给部分解，当给部分解的时候，gurobi内部会尝试着给出一个完整的解。
- `Linear Constraint Attributes`; 线性约束属性，像Pi约束对应的对偶值；slack约束的松弛量；RHS约束的右端项。这里常用的是Pi，在构造高级方法的时候，往往会用到对偶信息，像列生成和Benders分解。
- `Special-Ordered Set Constraints Attributes`; SOS约束条件，例如IISSOS对不可行的模型，指示约束是否属于IIS(Irreducible Inconsistent Subsystem)。
- `Quadratic Constraint Attributes`; 二次约束属性，像QCRHS约束右端项。
- `General Constraint Attributes`; 广义约束属性，例如GenConstrName约束名称。
- `Quality Attributes`; 解质量属性，例如像BoundVio最大的界违反：IntVio整数变量离最近整数的最大距离。
- `Multi-objective Attributes`; 多目标属性, 例如ObjN对应多目标表达式中变量系数：ObjNval对应目标函数值。
- `Multi-Scenario Attributes`;
- `Batch Attributes`; 

使用`.setAttr(attrname, newvalue)`来进行属性设置。attrname表示属性名称，newvalue表示属性的值。例如var.setAttr(GRB.Attr.VType, 'C')或简写为var.Vtype='C' 注: 并不是所有的属性都可以设置，如果某个属性不能调整的时候，就不能用这个来设置它的值。

使用`.getAttr(attrname, objs)`来查询属性设置。attrname表述属性名称，objs列表或字典对象用来存储查询的值。例如model.getAttr(GRB.Attr.ObjVal)或简写为model.ObjVal。

- 常用属性

|类别|属性名称|作用|取值|
|:-|:-|:-|:-|
|模型|ModelSense(可调整)|模型优化方向|1:最小化(默认); -1:最大化|
|模型|ObjVal(不可调整)|目标函数值|double|
|模型|Status(不可调整)|解的状态|1-15分别表示不同含义, 具体看refman|
|变量|LB/UB(可调整)|变量下界/上界|double|
|变量|Obj(可调整)|变量的线性目标系数|double|
|变量|VType(可调整)|变量类型|'C':continuous; <br>'B':Binary; 'I':Integer; <br>'S':Semi-Continuous; <br>'N':Semi-Integer|
|变量|X(不可调整)|变量值|double|
|变量|Start(可调整)|变量的初始值|double|
|约束|RHS(可调整)|线性约束的右端项|double|
|约束|Pi(不可调整)|线性约束对应的对偶变量|double|

In [3]:
import gurobipy as gp
from gurobipy import GRB
# Create Model
m = gp.Model()

# Create Vars
x = m.addVar(obj=1, vtype=GRB.CONTINUOUS, name='x')
y = m.addVar(obj=1, vtype=GRB.CONTINUOUS, name='y')
z = m.addVar(obj=1, vtype=GRB.CONTINUOUS, name='z')

# Write Model
m.update()
m.write('class2_Attribute_Example_1.lp')

# Set the vtype of vars
# Change the object of vars
x.vtype = 'B'  # 将x变量的连续变量类型调整为01变量整型。
x.obj=100  # 将x的目标值调整为100。
m.update()
m.write('class2_Attribute_Example_2.lp')

## 自动化参数调整

Gurobi提供了两种调参方式: 

常用调优参数: 

1. 通过命令行:

    grbtune TuneTimelimit=100 lp_file_path/mps_file_path
    
    
2. 通过API：
    
    model.tune()

`model.tune()`调参工具涉及到的一些参数如下: 

|参数名称|作用|取值|
|:-|:-|:-|
|tuneCriterion|调整调参准则|-1:默认, 缩短发现最优解所需的时间; <br>1:最优的Gap; <br>2:最好的可行解; <br>3:最好的bound|
|tuneJobs|分布式并行调参|0:默认|
|tuneOutPut|控制输出结果的量|0:没有输出; 1:发现最好参数组合时输出; 2:默认,输出试过的参数组合; 3:试过的参数组合和详细的求解器输出|
|tuneResults|返回最优参数组合的数量|-1:默认, 按照调整参数的个数返回调参结果|
|tuneTimelimit|调参时间|-1:默认, 自动选择时间|
|tuneTrails|每组参数组合运行的次数|主要目的减小随机因素的影响|

In [5]:
import gurobipy as gp
from gurobipy import GRB

# Load Model
model = gp.read('net12.lp')

# Set the TuneResults parameter to 1
model.Params.tuneResults = 1  # 调整参数的个数，返回调参结果。这里只返回一组。
model.Params.tuneTimeLimit = 20 # 调参时间限制。

# Tune the model
model.tune()  # 调用调参工具。

# 该案例因无更优参数，所以不会输出
# 若调参后发现了更好的参数组合
if model.tuneResultCount > 0:  # 大于0的时候，说明找到了更好的调参的组合。

    # Load the best tuned parameters into the model
    # index从0开始，第0个为最好的参数
    model.getTuneResult(0)  # 获得第几个参数组合。

    # Write tuned parameters to a file
    model.write('class2_tune.prm')  # 将调参的结果写到prm格式的文件中。

    # Solve the model using the tuned parameters
    model.optimize()

Read LP format model from file net12.lp
Reading time = 0.08 seconds
R14022: 14021 rows, 14115 columns, 80384 nonzeros
Changed value of parameter tuneResults to 1
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter tuneTimeLimit to 20.0
   Prev: -1.0  Min: -1.0  Max: inf  Default: -1.0

Solving model using baseline parameter set with TimeLimit=2s

Testing candidate parameter set 1...

	Default parameters

Solving with random seed #1 ...


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

## 广义约束的表达方式和使用

### 广义约束

&emsp;&emsp;第一部分有说如何去构建一些常规的约束，`Gurobi`也支持一些特殊的约束，比如像广义约束等。

&emsp;&emsp;广义约束约束的表达方式和使用:

1. 广义约束 - `Max` | 一组变量(包含常数)中取最大
    - `addGenConstrMax(resvar, vars, constant, name)`
        - resvar 变量(x = max(x1, x2, 10))
        - vars 一组变量(可以包含常数)
        - constant 常数
        - name 广义约束名称

&emsp;&emsp;例如:$z = max(x, y, 3)$。

```python
m.addGenConstrMax(z, [x, y], 3, "maxconstr")
m.addGenConstrMax(z, [x, y, 3], "maxconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(z == max_([x, y, 3]), "maxconstr")
m.addConstr(z == max_(x, y, 3), "maxconstr")
```

2. 广义约束 - `Min` | 一组变量(包含常数)中取最小
    - `addGenConstrMin(resvar, vars, constant, name)`
        - resvar 变量(x = min(x1, x2, 10))
        - vars 一组变量(可以包含常数)
        - constant 常数
        - name 广义约束名称

&emsp;&emsp;例如: $z = min(x, y, 3)$

```python
m.addGenConstrMin(z, [x, y], 3, "minconstr")
m.addGenConstrMin(z, [x, y, 3], name="minconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(z == min_([x, y, 3]), "minconstr")
m.addConstr(z == min_(x, y, 3), "minconstr")
```

3. 广义约束 - `Abs` | 取绝对值
    - `addGenConstrMin(resvar, argvars, name)`
        - resvar 变量
        - argvars 变量
        - name 广义约束名称

&emsp;&emsp;例如: $x = |y|$

```python
m.addGenConstrAbs(x, y, "absconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(x == abs_(y), "absconstr")
```

4. 广义约束 - `And` | 一组变量的值全等于1则取1，否则取0. 注: 所有输入的变量均被视为binary，不论之前被定义为什么类型。(注意：所有的变量都被视为0,1变量,不论他们之前被定为什么类型。)
    - `addGenConstrAnd(resvar, vars, name)`
        - resvar 变量
        - vars 变量
        - name 广义约束名称  

&emsp;&emsp;例如: $x = 1$且$y = 1$,那么$z = 1$,否则$z = 0$。

```python
m.addGenConstrAnd(z, [x,y], "andconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(z == and_(x, y), "andconstr")
```

5. 广义约束 - `Or` | 一组变量的值有一个等于1则取1，否则取0. 注: 所有输入的变量均被视为binary，不论之前被定义为什么类型
    - `addGenConstrAnd(resvar, vars, name)`
        - resvar 变量
        - vars 变量
        - name 广义约束名称 

&emsp;&emsp;例如: $x = 0$且$y = 0$,那么$z = 0$,否则$z = 1$。

```python
m.addGenConstrOr(z, [x,y], "orconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(z == or_(x, y), "orconstr")
```

6. 广义约束 - `Indicator` | 指示变量的值为1，约束成立，否则约束可被违反
    - `addGenConstrIndicator(binvar, binval, lhs, sense, rhs, name)`
        - binvar 指示变量
        - binval 指示变量的值(boolen)
        - lhs 约束左端项
        - sense 约束符号
        - rhs 约束右端项
        - name 广义约束名称    
        
&emsp;&emsp;例如: 如果$z = 1$, 则$x+y<= 4$。

```python
m.addGenConstrOr(z, [x,y], "orconstr")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(z == or_(x, y), "orconstr")
```

In [8]:
import gurobipy as gp
from gurobipy import GRB

# Create Model
m = gp.Model()

# Create Vars
x = m.addVar(vtype=GRB.CONTINUOUS, name='x')
y = m.addVar(vtype=GRB.CONTINUOUS, name='y')
z = m.addVar(vtype=GRB.CONTINUOUS, name='z')

# Set Objective
m.setObjective(x + y + z, GRB.MAXIMIZE)

# MAX
m.addGenConstrMax(z, [x, y, 3], name='maxconstr')
# MIN
m.addGenConstrMin(z, [x, y, 3], name='minconstr')
# ABS
m.addGenConstrAbs(z, y, name='absconstr')
# AND
m.addGenConstrAnd(z, [x, y], name='andconstr')
# OR
m.addGenConstrOr(z, [x, y], name='orconstr')
# INDICATOR
m.addGenConstrIndicator(z, True, x+y, GRB.LESS_EQUAL, 4, 'indicatorconstr')

# Write
m.write('class2_GenConstr_Example.lp')

### 范围约束

&emsp;&emsp;其他特殊约束的表达方式和使用: 

- 范围约束
    - `addRange(expr, lower, upper, name)` | 一个表达式需要在一定范围内
        - expr 表达式
        - lower 下界
        - upper 上界
        - name 约束名称

&emsp;&emsp;例如: $5 <= x + y + z <= 10$。

```python
m.addRange(x+y+z, 5, 10, "c")
```

&emsp;&emsp;换成一般约束的表达方式的话，就需要写成如下形式:

```python
m.addConstr(x+y+z==[5,12])
```

- SOS(Special-Ordered Set)约束 | 
    - addSOS(type, vars, wts=None)
        - type 约束种类(GRB.SOS_TYPE1 或者 GRB.SOS_TYPE2)
            - GRB.SOS_TYPE1 表示一组有序变量中**最多有一个变量取值不为0**。
            - GRB.SOS_TYPE2 表示一组有序变量中**最多有两个变量取值不为0,且非零变量相邻。变量是否相邻由权重决定**。
        - vars 变量
        - wts 变量对应的权重, 且权重唯一, 默认1,2,3.....

```python
m.addSOS(GRB.SOS_TYPE2, [x, y, z], [1, 2, 4])
```


In [10]:
import gurobipy as gp
from gurobipy import GRB

# Create Model
m = gp.Model()

# Create Vars
x = m.addVar(vtype=GRB.CONTINUOUS, name='x')
y = m.addVar(vtype=GRB.CONTINUOUS, name='y')
z = m.addVar(vtype=GRB.CONTINUOUS, name='z')

# Set Objective
m.setObjective(x + y + z, GRB.MINIMIZE)

# RANGE
m.addRange(x+y+z, 5, 10, 'c')
# SOS
# SOS1 - x与y相邻，y与z相邻
# 不给权重时，默认列表内顺序如何就是如何相邻的
m.addSOS(GRB.SOS_TYPE1, [x,y,z])
# SOS2 - y与z相邻，z与x相邻
# 给予了权重，所以不按列表内顺序进行相邻
# m.addSOS(GRB.SOS_TYPE2, [x,y,z], [1,4,2])

# Write
m.optimize()

# Print result
for v in m.getVars():
    print('{} {}'.format(v.varName, v.x))
print('Obj: {}'.format(m.objVal))

# 运行的结果为[x,y,z]中的一个值等于5，其余值均为0; 因为 [x,y,z] 为SOS1, 仅有一个可以为非0

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x38ceb361
Model has 1 SOS constraint
Variable types: 4 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+00]
  RHS range        [1e+01, 1e+01]
Presolve added 3 rows and 2 columns
Presolve time: 0.02s
Presolved: 4 rows, 6 columns, 11 nonzeros
Variable types: 4 continuous, 2 integer (2 binary)

Root relaxation: objective 5.000000e+00, 1 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       5.0000000    5.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.11 seconds
Thread count was 4 (of 4 available processo

## 多目标约束的表达方式和使用

- 目标函数(多个目标) 
    - `setObjectiveN(expr, index, priority, weight, abstol, reltol, name)`里面主要有7个参数。
        - expr 目标函数表达式
        - index 目标函数对应的序号(0,1,2,...)
        - priority 优先级(整数值)
        - weight 权重(浮点数)
        - abstol 允许的目标函数值最大的降低值(浮点数)
        - reltol 允许的目标函数值最大的降低至reltol*|目标函数值|(浮点数)
        - name 目标函数名
        - 注: 所有的**目标函数均需为线性的**，且目标函数的优化方向一致(全最大化或全最小化)。若优化方向不一致,可以通过乘'-1'实现不同的优化方向
    
    - 可以通过参数ObjNumber选择特定的目标，进而获得对应的目标函数值

### 多目标优化

Gurobi支持三种多目标模式:

1. Blend(合成型) | 有权重，没有优先级

    Obj1 = x + y, weight1 = 1
    
    Obj2 = x - 5y, weight2 = -2

    Gurobi会混合这两个目标值形成 -> 1*(x+y) - 2*(x-5y) -> -x+11y

2. Hierarchial(分层性) | 有优先级

    Obj1 = x + y, priority1 = 10
    
    Obj2 = x - 5y, priority2 = 5
    
    Gurobi按照优先级大小优化(先优化Obj1), 若没有设定abstol或reltol, 在优化低优先级目标时, 不会改变高优先级的目标值。
    
    假设Obj1=5, 在优化Obj2时只能在是的Obj1=5的所有解中挑选最优解(当未设置abstol与reltol时)

3. 混合以上两种

1. Blend

```python
    m.setObjectiveN(x+y+3*z, index=0, weight=0.1, name='obj1')
    m.setObjectiveN(2*x+y+3*z, index=1, weight=3, name='obj2')
```

2. Hierarchial

```python
    m.setObjectiveN(x+y+3*z, index=0, priority=1, name='obj1')
    m.setObjectiveN(2*x+y+3*z, index=1, priority=3, name='obj2')
```

3. 混合以上两种

```python
    m.setObjectiveN(x+y+3*z, index=0, weight=0.1, priority=1, name='obj1')
    m.setObjectiveN(2*x+y+3*z, index=1, weight=3, priority=3, name='obj2')
```

&emsp;&emsp;如果想要查看多个目标的目标函数取值，通过参数`ObjNumber`选择特定的目标，进而获得对应的目标函数值

```python
for i in range(model.NumObj):
    modle.setParam(GRB.Param.ObjNumber, i)
    print('Obj%d =' %(i+1), model.ObjNVal)
```

### 多目标案例(多目标指派问题)

&emsp;&emsp;假设工厂需要把$N$项工作分配给$N$个工人, 且每项工作只能由一个工人做, 每位工人也只能做一项工作。已知工人$i(i \in N)$处理工作$j(j \in N)$需要时间$T_{ij}$, 获得的利润为$C_{ij}$。找出一种安排方案使得完成所有工作需要的总时间最短且获得的总利润最大。
   
$$obj1: min \sum_{i=1}^{N} \sum_{j=1}^{N} T_{ij}x_{ij} \\
obj2: max \sum_{i=1}^{N} \sum_{j=1}^{N} C_{ij}x_{ij} \\
\sum_{i=1}^{N}x_{ij}=1, \forall j \in N \\
\sum_{j=1}^{N}x_{ij}=1, \forall i \in N \\
x_{ij} \in {0,1}, \forall i,j \in N
$$

In [11]:
import gurobipy as gp
import random
from gurobipy import GRB


# 设定工作与工人数量； 还有种子值
N = 20
random.seed(1)

# 产生随机成本和时间矩阵 Tmatrix 和 Cmatrix
Tmatrix = {(i+1,j+1):random.randint(0,100) for i in range(N) for j in range(N)}
Cmatrix = {(i+1,j+1):random.randint(0,100) for i in range(N) for j in range(N)}

# 定义 model
m = gp.Model('class2_MultiAssignment')

# 添加变量
x = m.addVars(Tmatrix.keys(), vtype=GRB.BINARY, name='x')

# 添加约束
m.addConstrs((x.sum('*',j+1) == 1 for j in range(N)),'C1')
m.addConstrs((x.sum(i+1,'*') == 1 for i in range(N)),'C2')
 
# 设置多目标 权重
# 所有目标函数，默认求min
# 总和就是目标函数为 min(0.1*Time - 0.5*Cost)
m.setObjectiveN(x.prod(Tmatrix),  index=0, weight=0.1, name='obj1')
m.setObjectiveN(-x.prod(Cmatrix), index=1, weight=0.5, name='obj2')

# # 设置多目标 优先级
# m.setObjectiveN(x.prod(Tmatrix),  index=0,  priority=1, abstol=0, reltol=0, name='obj1')
# m.setObjectiveN(-x.prod(Cmatrix), index=1,  priority=2, abstol=100, reltol=0, name='obj2')

# 设置logFile的名称
m.setParam(GRB.Param.LogFile, 'class2_MultiAssignmentLog.log')

# 启动求解
m.optimize()

# 获得求解结果
for i in Tmatrix.keys():
    if x[i].x>0.9:
        print ("工人 %d \t ----> \t 工作 %d" %(i[0], i[1]))

for i in range(2):
    m.setParam(GRB.Param.ObjNumber, i)
    print('Obj%d = '%(i+1), m.ObjNVal)

#将model输出到 lp格式文件中
m.write('class2_MultiAssinment.lp')

Changed value of parameter LogFile to class2_MultiAssignmentLog.log
   Prev:   Default: 
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 40 rows, 400 columns and 800 nonzeros
Model fingerprint: 0xf3ec5400
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 40 rows, 400 colu

### 分段线性目标

- 目标函数(分段线性目标)

- setPWLObj(var, x, y) | **对一些非线性模型，可以使用这一功能去逼近**。
    - var 指定变量的目标函数是分段线性
    - x 定义分段线性目标函数的点的横坐标值(非减序列)
    - y 定义分段线性目标函数的点的纵坐标值

&emsp;&emsp;`Example1`，比如有这样一个目标函数:

$$
Max \ f(x) = 
\begin{cases} 
x \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (0 <= x <= 2) \\
\frac{1}{3}x + \frac{4}{3} \ \ \ \ \ \ \ \ (2 <= x <= 5)
\end{cases}
$$

&emsp;&emsp;相当于取了$(0,0)、(2,2)、(5,3)$三个点, 然后自动将这三个点用两条线段相连

```python
m.setPWLObj(x, [0,2,5], [0,2,3])
m.setAttr(GRB.Attr.ModelSense, -1)
```

In [12]:
# Create Model
m = gp.Model()

# Create Vars
x = m.addVar(lb=0, ub=5, vtype=GRB.CONTINUOUS, name='x')

# PWL
m.setPWLObj(x, [0, 2, 5], [0, 2, 3])

# 设置模型求最大值(若不设置则默认为求最小值)
# 具体参见1.3 Attribute
m.setAttr(GRB.Attr.ModelSense, -1)

#启动求解
m.optimize()

#获得求解结果
print('Obj: %g' % m.ObjVal)
for v in m.getVars():
    print('%s %f' % (v.VarName, v.X))

# write file
m.write('class2_PiecewiseLinear_Example_1.lp')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x9e96c4d2
Model has 1 piecewise-linear objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [5e+00, 5e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.03s
Presolved: 0 rows, 1 columns, 0 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds
Optimal objective  3.000000000e+00
Obj: 3
x 5.000000


&emsp;&emsp;再看一个例子:

&emsp;&emsp;Example2: 使用分段目标逼近非线性(其实该模型为二阶线性，可以直接求的，只不过这里使用线性模型去做逼近这件事)

$$min \ f(x) = (x-1)^2 + x, x\in [0,10]$$

In [13]:
import gurobipy as gp
from gurobipy import GRB

try:
    npts = 10000  # 设定分割的密度。
    
    m = gp.Model()  # 定义model               
    
    x = m.addVar(lb=0.0, ub=10.0, vtype=GRB.CONTINUOUS, name='x')  # 添加变量, 变量下界为0，上界为10。
    
    #定义分割点
    px = []
    f = []
    for i in range(npts):
        px.append(10 * i / (npts - 1))
        f.append((px[i]*px[i])-2*px[i] + 3)
    m.setPWLObj(x, px, f)
    
    #启动求解
    m.optimize()
    
    #获得求解结果
    print('Obj: %g' % m.ObjVal)
    for v in m.getVars():
        print('%s %f' % (v.VarName, v.X))
    
    m.write("class2_PiecewiseLinear_Example_2.lp")

except GurobiError:
    print('Error reported') 

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x24c2bb8d
Model has 1 piecewise-linear objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 0 rows, 1 columns, 0 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds
Optimal objective  2.000000010e+00
Obj: 2
x 1.000100


## 多个解集合的实现方式和使用

Gurobi在搜寻最优解过程中，会找到一些次优解(sub-optimal solutions), 有时候用户也希望知道次优解的具体情况。

Gurobi会将计算过程中发现的所有解记录在Solution Pool里供用户查询:

|参数名称|作用|取值|
|:-|:-|:-|
|PoolSearchMode|发现解的方法|0:默认。搜寻最优解; <br>1:搜寻更多的解但是不保证解的质量; <br>2:尝试寻找n个解(n由PoolSolutions设定)|
|PoolGap|设定Pool的Gap值|Infinity, 默认。所有解与最优解的gap小于设定的值|
|PoolSolutions|设定Pool存放解的数量|10, 默认。设定pool里储存解的个数|

Solution Pool里的解按照质量非增排序，并从零开始编号。因此，查询具体解的情况(变量值和对应目标值等)，需要先设定SolutionNumber的值，然后通过模型属性PoolObjVal和变量属性Xn获得目标值和变量值


例如: 查询Pool里面第四个解的目标值和变量值

```python
model.setparam(GRB.Param.SolutionNumber, 3)
print('obj=', model.PoolObjVal)
for i in range(model.NumVars):
    print(Vars[i].VarName, '=', Vars[i].Xn)
```

In [14]:
from gurobipy import *

try:
    # Sample data
    Groundset = range(10)
    objCoef = [32, 32, 15, 15, 6, 6, 1, 1, 1, 1]
    knapsackCoef = [16, 16, 8, 8, 4, 4, 2, 2, 1, 1]
    Budget = 33

    # Create initial model
    model = Model("poolsearch")

    # Create dicts for tupledict.prod() function
    objCoefDict = dict(zip(Groundset, objCoef))
    knapsackCoefDict = dict(zip(Groundset, knapsackCoef))

    # Initialize decision variables for ground set:
    # x[e] == 1 if element e is chosen
    Elem = model.addVars(Groundset, vtype=GRB.BINARY, name='El')

    # Set objective function
    model.ModelSense = GRB.MAXIMIZE
    model.setObjective(Elem.prod(objCoefDict))

    # Constraint: limit total number of elements to be picked to be at most
    # Budget
    model.addConstr(Elem.prod(knapsackCoefDict) <= Budget, name='Budget')

    # Limit how many solutions to collect
    model.setParam(GRB.Param.PoolSolutions, 1024)  # 设置保存解的数量。
    # Limit the search space by setting a gap for the worst possible solution that will be accepted。
    model.setParam(GRB.Param.PoolGap, 0.10)  # 设置解的gap，如果解的gap大于0.1的话，就不会存放在这里。
    # do a systematic search for the k-best solutions
    model.setParam(GRB.Param.PoolSearchMode, 2)  # 设定一下发现解的方向，这里我们期望gurobi去寻找1024个解。

    # save problem
    model.write('poolsearch.lp')

    # Optimize
    model.optimize()

    model.setParam(GRB.Param.OutputFlag, 0)

    # Status checking
    status = model.Status
    if status == GRB.Status.INF_OR_UNBD or \
       status == GRB.Status.INFEASIBLE or \
       status == GRB.Status.UNBOUNDED:
        print('The model cannot be solved because it is infeasible or unbounded')
        sys.exit(1)

    if status != GRB.Status.OPTIMAL:
        print('Optimization was stopped with status ' + str(status))
        sys.exit(1)

    # Print best selected set
    print('Selected elements in best solution:')
    print('\t', end='')
    for e in Groundset:
        if Elem[e].X > .9:
            print(' El%d' % e, end='')
    print('')

    # Print number of solutions stored
    nSolutions = model.SolCount
    print('Number of solutions found: ' + str(nSolutions))

    # Print objective values of solutions
    for e in range(nSolutions):
        model.setParam(GRB.Param.SolutionNumber, e)
        print('%g ' % model.PoolObjVal, end='')
        if e % 15 == 14:
            print('')
    print('')

    # print fourth best set if available
    if (nSolutions >= 4):  # 查询SolutionPool里面第四个解的具体信息。
        model.setParam(GRB.Param.SolutionNumber, 3)
        print('Selected elements in fourth best solution:')
        print('\t', end='')
        for e in Groundset:
            if Elem[e].Xn > .9:
                print(' El%d' % e, end='')
        print('')

except GurobiError as e:
    print('Gurobi error ' + str(e.errno) + ": " + str(e.message))

except AttributeError as e:
    print('Encountered an attribute error: ' + str(e))
    
    
# 在设置gap为0.1的情况下，Gurobi一共找到21各解
# 其中第四个解，中El1 El2 El3 El8为1，其他均为0

Changed value of parameter PoolSolutions to 1024
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolGap to 0.1
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x6d63cbfb
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 65.0000000
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.587500e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds     